<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/04-다중_언어_개체명_인식.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  텍스트 분류 목차
* [Chapter 0 개요](#chapter0)
* [Chapter 1 데이터셋](#chapter1)

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. NLP를 적용할 문서가 다른 언어이거나, 다국어로 되어있는 경우 문제가 발생할 수 있다.
    - 러시아어, 중국어, 독일어와 같은 대표적인 언어는 허깅페이스에서 적절한 사전 훈련된 언어 모델을 찾아 미세튜닝 가능하다.
    - 그리스어, 스와힐리어 등의 언어는 사전 훈련된 모델을 찾기 어려울 수 있다.

2. 다중 언어 트랜스포머 모델이 등장하면서, 하나의 모델로 여러 언어를 처리할 수 있게 되었다.
    - 대표적인 다중 언어 트랜스포머 모델: mBERT, XLM-RoBERTa
    - 다중 언어 트랜스포머 모델은 여러 언어로 된 대규모 말뭉치를 사용해 사전 훈련되었다.
    - 다중 언어 트랜스포머 모델은 단일 언어 모델에 비해 성능이 약간 떨어질 수 있지만, 다양한 언어를 처리할 수 있다는 장점이 있다.

3. 다중 언어 트랜스포머는 많은 언어로 된 대규모 말뭉치에서 사전 훈련해서 제로샷 교차 언어 전이가 가능하다.
    - 제로샷 교차 언어 전이: 모델이 훈련되지 않은 언어로 된 문서에 대해 예측할 수 있는 능력
    - 예를 들어, 영어로 된 데이터셋으로 미세튜닝한 모델이 러시아어로 된 문서에 대해서도 예측할 수 있다.

4. 이번 장에서는 XML-RoBERTa 모델을 사용해 다중 언어 개체명 인식(NER) 작업을 수행한다.
    - 개체명 인식(NER): 문서에서 사람, 장소, 조직 등의 개체를 식별하는 작업
    - 다중 언어 NER 작업에서는 여러 언어로 된 문서에서 개체를 식별해야 한다.
    - 예를 들어 회사 문서에서 중요한 정보를 추출하거나, 검색 엔진의 품직을 높이거나, 말뭉치에서 구조적인 데이터를 생성하는 데 활용할 수 있다.

5. 네 개의 공용어를 사용하여 스위스에서 주로 활동하는 고객을 위해 NER을 수행한다고 가정한다.

## Chapter 1 데이터셋 <a class="anchor" id="chapter1"></a>
1. WikiANN 또는 APN-X라 불리는 교차 언어 전이 평가 벤치마크 데이터를 사용한다.
    - 여러 언어의 위키피디피아 문서로 구성되고 스위스에서 사용되는 독일어(62.9%), 프랑스어(22.9%), 이탈리아어(8.4%), 영어(5.9%) 네 개 언어로 작성됐다.
    - 각 문서는 IOB2 포멧으로 LOC(위치), PER(사람), ORG(조직) 세 가지 개체 유형으로 주석이 달려 있다.
        - B- 접두사: 개체명의 시작을 나타낸다.
        - I- 접두사: 동릴한 개체명에 속해 연속되는 토큰을 나타낸다.
        - O: 개체명에 속하지 않는 토큰을 나타낸다.
    - 예를 그림으로 표현하면 아래와 같다.

         ![WikiANN 예시](image/04_01_WikiANN.png)


In [2]:
# TMREME에서 PAN-X 서브셋 중 하나를 로드한다.
#   - load_dataset() 함수에 전달할 이름을 확인하기 위해 get_dataset_config_names() 함수를 사용한다.
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME 서브셋 개수: {len(xtreme_subsets)}")
print(xtreme_subsets)

XTREME 서브셋 개수: 183
['MLQA.ar.ar', 'MLQA.ar.de', 'MLQA.ar.en', 'MLQA.ar.es', 'MLQA.ar.hi', 'MLQA.ar.vi', 'MLQA.ar.zh', 'MLQA.de.ar', 'MLQA.de.de', 'MLQA.de.en', 'MLQA.de.es', 'MLQA.de.hi', 'MLQA.de.vi', 'MLQA.de.zh', 'MLQA.en.ar', 'MLQA.en.de', 'MLQA.en.en', 'MLQA.en.es', 'MLQA.en.hi', 'MLQA.en.vi', 'MLQA.en.zh', 'MLQA.es.ar', 'MLQA.es.de', 'MLQA.es.en', 'MLQA.es.es', 'MLQA.es.hi', 'MLQA.es.vi', 'MLQA.es.zh', 'MLQA.hi.ar', 'MLQA.hi.de', 'MLQA.hi.en', 'MLQA.hi.es', 'MLQA.hi.hi', 'MLQA.hi.vi', 'MLQA.hi.zh', 'MLQA.vi.ar', 'MLQA.vi.de', 'MLQA.vi.en', 'MLQA.vi.es', 'MLQA.vi.hi', 'MLQA.vi.vi', 'MLQA.vi.zh', 'MLQA.zh.ar', 'MLQA.zh.de', 'MLQA.zh.en', 'MLQA.zh.es', 'MLQA.zh.hi', 'MLQA.zh.vi', 'MLQA.zh.zh', 'PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de', 'PAN-X.el', 'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu', 'PAN-X.fa', 'PAN-X.fi', 'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu', 'PAN-X.id', 'PAN-X.it', 'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk', 'PAN-X.ko', 'PAN-X.ml', 'PAN-X

In [3]:
# "PAN"으로 시작하는 서브셋을 찾는다.
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
print(f"PAN-X 서브셋 개수: {len(panx_subsets)}")
print(panx_subsets)

PAN-X 서브셋 개수: 40
['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de', 'PAN-X.el', 'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu', 'PAN-X.fa', 'PAN-X.fi', 'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu', 'PAN-X.id', 'PAN-X.it', 'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk', 'PAN-X.ko', 'PAN-X.ml', 'PAN-X.mr', 'PAN-X.ms', 'PAN-X.my', 'PAN-X.nl', 'PAN-X.pt', 'PAN-X.ru', 'PAN-X.sw', 'PAN-X.ta', 'PAN-X.te', 'PAN-X.th', 'PAN-X.tl', 'PAN-X.tr', 'PAN-X.ur', 'PAN-X.vi', 'PAN-X.yo', 'PAN-X.zh']


2. ISO 639-1 언어 코드로 보이는 두 문자로 된 접미사가 있다.
    - de: 독일어
    - fr: 프랑스어
    - it: 이탈리아어
    - en: 영어

In [4]:
from datasets import load_dataset
panx_de = load_dataset("xtreme", "PAN-X.de")
print(panx_de)

PAN-X.de/train-00000-of-00001.parquet:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

PAN-X.de/validation-00000-of-00001.parqu(…):   0%|          | 0.00/590k [00:00<?, ?B/s]

PAN-X.de/test-00000-of-00001.parquet:   0%|          | 0.00/588k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})


3. 독일어, 프랑스어, 이탈리아어, 영어 말뭉치를 샘를링한다.
    - 분균형한 데이터셍이 만들어지는데, 실제 데이터셋에서는 흔히 벌어지는 일이다.
    - 소수 언어에서 레이블링된 샘플을 구하려면 비용이 많이 들기도 한다. 

In [11]:
from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]  # PAN-X 언어별 샘플 비율

panx_ch = defaultdict(DatasetDict) # 키가 없는 경우 빈 DatasetDict 생성

for lang, frac in zip(langs, fracs):
    # 다국어 말뭉치를 로드합니다.
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    # 각 분할을 언어 비율에 따라 다운샘플링하고 섞습니다.
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows))))

In [12]:
# 훈련 세트에 언어마다 얼마나 많은 샘플이 들어있는지 확인한다.
import pandas as pd

pd.DataFrame({lang: panx_ch[lang]["train"].num_rows for lang in langs}, index=["훈련 샘플 수"])

,de,fr,it,en
훈련 샘플 수,12580,4580,1680,1180


4. 독일어 샘플은 그 외 언어를 모두 합친 것보다 더 많다.

5. 이 데이터셋을 사용해 제로샷 교차 언어 전이를 프랑스어, 이탈리아어, 영어에 수행한다.

In [13]:
# 독일어 말뭉치에 있는 샘플 한 개 확인
#   - 샘플의 키는 애로우 테이블의 열 이름에 해당하고 값은 각 열에 있는 항목이다.
#   - ner_tags 열은 각 개체면에 매핑된 클래스 ID에 해당한다.
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [ ]:
# 이해하기 쉽게 LOC, PER, ORG 태크로 새로운 열을 만든다.
#   - Dataset 객체는 각 열의 데이터 타입을 담은 feature 속성을 가진다.
#   - nert_tags 열은 ClassLable의 리스트이다.
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: List(Value('string'))
ner_tags: List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))
langs: List(Value('string'))


In [16]:
# 훈련 세트에서 특성을 확인한다
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])


In [19]:
# int2str() 메서드를 사용해 클래스 ID를 레이블 문자열로 변환한다.
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(i) for i in batch["ner_tags"]]}

panx_de = panx_ch["de"].map(create_tag_names, batched=True)

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

In [20]:
# 첫 번째 샘플의 토큰과 태그 이름을 나란히 출력한다.
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]], index=["토큰", "태그"])

,0,1,2,3,4,5,6,7,8,9,10,11
토큰,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
태그,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [ ]:
# 태그가 분균형하게 부여되지 않았나 확인하기 위해 각 분할에서 개체면의 빈도를 계산한다.
#  - B-LOC, B-PER, B-ORG 태그로 시작하는 태그의 개수를 센다.
from collections import Counter

split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index") 

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071
